In [17]:
!pip install datasets
!pip install -qq emoji

In [18]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
import string
from keras.utils import np_utils
from tensorflow.keras import layers
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import json
from pandas.io.json import json_normalize
from scipy import stats
import emoji as emoji
import datasets
from datasets import load_dataset

In [19]:
import datasets
from datasets import load_dataset

hatexplain_ds = load_dataset("hatexplain")

Reusing dataset hatexplain (/root/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']
   

def flatten(t):
    return [item for sublist in t for item in sublist]

def pre_process_sentence(value):
    text = value
    for emo in text:
      if emo in emoji.UNICODE_EMOJI:
            text = text.replace(emo, "<emoticon>")
    for emo in emoticons:
        text = text.replace(emo, "<emoticon>")
    return text

def modify_record( record):
  mode = stats.mode(record ['annotators']['label'])
  label = mode[0][0]
  if(label == 2):
    label = 1
  elif label ==1:
    label = 2  
  record['label'] = label
  record['votes'] = mode[1][0]
  record['sentence'] = pre_process_sentence(' '.join(record['post_tokens']))
  record['token_count'] = len(record['post_tokens'])
  targets = flatten(record ['annotators']['target'])
  record['target'] = stats.mode(targets)[0][0]
  return record

In [21]:
tags = set()
def check_tags(record):
  for token in record['post_tokens']:
    if(token.startswith('<')):
      tags.add(token)

hatexplain_ds.map(check_tags)

print(tags)


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

{'<sad>', '<censored>', '<email>', '<url>', '<wink>', '<happy>', '<annoyed>', '<tong>', '<number>', '<money>', '<phone>', '<percent>', '<kiss>', '<angel>', '<date>', '<time>', '<user>', '<surprise>', '<will>', '<laugh>'}


In [22]:
hatexplain_updated_ds = hatexplain_ds.map(modify_record)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [23]:
hatexplain_merged_ds = concatenate_datasets([hatexplain_updated_ds['train'],hatexplain_updated_ds['validation'],hatexplain_updated_ds['test']])

In [26]:
hatexplain_merged_ds

Dataset({
    features: ['id', 'annotators', 'rationales', 'post_tokens', 'label', 'votes', 'sentence', 'token_count', 'target'],
    num_rows: 19229
})

In [25]:
hatexplain_merged_ds.save_to_disk('/content/drive/MyDrive/Dissertation/hateexplain_merged_ds')